## Installation and Import

In [1]:
!pip install tkmacosx

In [2]:
# for antispoofing module
!pip install easydict
!pip install numpy
!pip install tqdm
!pip install torchvision
!pip install torch
!pip install opencv-python
!pip install Pillow
!pip install tensorboardX

In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import load_model
import tkinter as tk
from tkmacosx import Button
from tkinter import messagebox
from PIL import Image, ImageTk

 # for recording attendance
import csv
from datetime import datetime, timedelta

# for antispoof
import sys

# Assuming 'anti_spoof_src' directory is in the same directory as the notebook
project_dir = os.path.abspath('antispoofing')
if project_dir not in sys.path:
    sys.path.append(project_dir)
    
from antispoofing.test import test

## Loading L1Dist and Model

In [4]:
# Siamese L1 Distance class
# pass in keras layer as param
class L1Dist(Layer):
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()

    # input_embedding is anchor, validaation_embedding is either positive or negative
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(tf.convert_to_tensor(input_embedding) - tf.convert_to_tensor(validation_embedding)) # difference (similarity calculation)# Siamese L1 Distance class

In [5]:
siamese_model = load_model('siamesemodelv2.h5', custom_objects={'L1Dist': L1Dist, 'BinaryCrossentropy': tf.losses.BinaryCrossentropy})

2024-05-27 20:15:59.601392: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-05-27 20:15:59.601416: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-05-27 20:15:59.601420: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-05-27 20:15:59.601609: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-27 20:15:59.601625: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
# Manually compile the model
siamese_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.Precision(name='precision'), tf.keras.metrics.Recall(name='recall')]
)

## Preprocessing

In [7]:
# Preprocess the image
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (100, 100))
    img = img / 255.0
    return img

## Verify Function, Register Function and Writing Attendance

In [8]:
# Verify function
def verify(model, input_img, verification_images_folder):
    results = []
    for person in os.listdir(verification_images_folder):
        if person == '.DS_Store':
            continue
        person_folder = os.path.join(verification_images_folder, person)
        for image in os.listdir(person_folder):
            validation_img = preprocess(os.path.join(person_folder, image))
            result = model.predict(list(tf.expand_dims([input_img, validation_img], axis=1)))
            print(result, person)
            results.append((result, person))
    
    detection_threshold = 0.5
    detection = [res[0] > detection_threshold for res in results]
    verification = np.sum(detection) / len(detection)
    
    if verification > 0.5:
        return True, max(results, key=lambda x: x[0])[1]
    else:
        return False, None

In [9]:
def log_event(person_name, event):
    with open('siamese_application_data/attendance.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([person_name, event, datetime.now()])

In [10]:
def register_face(name):
    user_dir = os.path.join('siamese_application_data', 'verification_images_app', name)
    os.makedirs(user_dir, exist_ok=True)
    
    cap = cv2.VideoCapture(1)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    count = 0
    
    while count < 10:
        ret, frame = cap.read()
        if not ret:
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            face_img = cv2.resize(face_img, (100, 100))
            cv2.imwrite(os.path.join(user_dir, f'{name}_{count}.jpg'), face_img)
            count += 1
            if count >= 10:
                break
    
    cap.release()

## UI Functions

In [11]:
def get_button(window, text, color, command, fg='white'):
    button = Button(
                        window,
                        text=text,
                        activebackground="black",
                        activeforeground="white",
                        fg=fg,
                        bg=color,
                        command=command,
                        height=100,
                        width=400,
                        font=('Helvetica bold', 20)
                    )
    return button

def get_img_label(window):
    label = tk.Label(window)
    label.grid(row=0, column=0)
    return label

def get_text_label(window, text):
    label = tk.Label(window, text=text)
    label.config(font=("sans-serif", 21), justify="left")
    return label

def get_entry_text(window):
    inputtxt = tk.Text(window, height=2, width=15, font=("Arial", 32))
    return inputtxt

def msg_box(title, description):
    tk.messagebox.showinfo(title, description)

## Main App

In [ ]:
class App:
    def __init__(self):
        self.main_window = tk.Tk()
        self.main_window.geometry("1200x520+350+100")
        self.main_window.title("Face Attendance System")

        self.text_label_register_new_user = get_text_label(self.main_window, 'Face Attendance System \n by Zun Yang Chin')
        self.text_label_register_new_user.place(x=750, y=70)

        self.login_button_main_window = get_button(self.main_window, 'Login', 'green', self.login)
        self.login_button_main_window.place(x=750, y=200)

        self.logout_button_main_window = get_button(self.main_window, 'Logout', 'red', self.logout)
        self.logout_button_main_window.place(x=750, y=300)

        self.register_new_user_button_main_window = get_button(self.main_window, 'Register New User', 'blue', self.register_new_user)
        self.register_new_user_button_main_window.place(x=750, y=400)

        self.webcam_label = get_img_label(self.main_window)
        self.webcam_label.place(x=10, y=0, width=700, height=500)

        self.db_dir = './siamese_application_data/verification_images_app'
        if not os.path.exists(self.db_dir):
            os.mkdir(self.db_dir)

        self.webcam_update_id = None
        self.cap = cv2.VideoCapture(1)
        self.add_webcam(self.webcam_label)

        # Bind the close event to the main window
        self.main_window.protocol("WM_DELETE_WINDOW", self.on_closing)

    def add_webcam(self, label):
        self._label = label
        self.process_webcam()

    def process_webcam(self):
        ret, frame = self.cap.read()
        if not ret:
            return

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        self.box_color = (255, 0, 0)  # Always blue

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), self.box_color, 2)
            self.most_recent_capture_arr = frame[y:y + h, x:x + w]

        img_ = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        self.most_recent_capture_pil = Image.fromarray(img_)
        self.imgtk = ImageTk.PhotoImage(image=self.most_recent_capture_pil)
        self._label.imgtk = self.imgtk
        self._label.configure(image=self.imgtk)

        self.full_frame = frame  # Store the full frame

        self.webcam_update_id = self._label.after(20, self.process_webcam)

    def cancel_webcam_update(self):
        if self.webcam_update_id:
            self._label.after_cancel(self.webcam_update_id)
            self.webcam_update_id = None

    def login(self):
        self.cancel_webcam_update()
        input_img_path = 'siamese_application_data/input_image/input_image.jpg'
        cv2.imwrite(input_img_path, self.most_recent_capture_arr)

        # Perform anti-spoofing check with the full frame
        label = test(
            image=self.full_frame,
            model_dir='/Users/zunyang/Library/CloudStorage/OneDrive-SwinburneUniversityOfTechnologySarawakCampus/Year2Sem2/AML/Face_Recognition_Project/antispoofing/resources/anti_spoof_models',
            device_id=0
        )

        if label == 1:
            input_img = preprocess(input_img_path)
            last_verified, last_person_name = verify(siamese_model, input_img, self.db_dir)

            if last_verified:
                msg_box('Login Successfully!', 'Welcome, {}.'.format(last_person_name))
                log_event(last_person_name, 'login')
            else:
                msg_box('Ups...', 'Unknown user. Please register new user or try again.')
        else:
            msg_box('You are fake!', 'Fake face detected. Try again.')

        self.add_webcam(self.webcam_label)

    def logout(self):
        self.cancel_webcam_update()
        input_img_path = 'siamese_application_data/input_image/input_image.jpg'
        cv2.imwrite(input_img_path, self.most_recent_capture_arr)

        # Perform anti-spoofing check with the full frame
        label = test(
            image=self.full_frame,
            model_dir='/Users/zunyang/Library/CloudStorage/OneDrive-SwinburneUniversityOfTechnologySarawakCampus/Year2Sem2/AML/Face_Recognition_Project/antispoofing/resources/anti_spoof_models',
            device_id=0
        )

        if label == 1:
            input_img = preprocess(input_img_path)
            last_verified, last_person_name = verify(siamese_model, input_img, self.db_dir)

            if last_verified:
                msg_box('Log Out Successfully!', 'Goodbye, {}.'.format(last_person_name))
                log_event(last_person_name, 'logout')
            else:
                msg_box('Ups...', 'Unknown user. Please register new user or try again.')
        else:
            msg_box('You are fake!', 'Fake face detected. Try again.')

        self.add_webcam(self.webcam_label)

    def register_new_user(self):
        self.cancel_webcam_update()
        self.main_window.withdraw()  # Hide the main window

        self.register_new_user_window = tk.Toplevel(self.main_window)  # Create a new Toplevel window for registration
        self.register_new_user_window.geometry("1200x520+370+120")
        self.register_new_user_window.title("Register New User")

        self.register_button = get_button(self.register_new_user_window, 'Register', 'green', self.accept_register_new_user)
        self.register_button.place(x=750, y=300)

        self.cancel_button = get_button(self.register_new_user_window, 'Cancel', 'red', self.cancel_register)
        self.cancel_button.place(x=750, y=400)

        self.capture_label = get_img_label(self.register_new_user_window)
        self.capture_label.place(x=10, y=0, width=700, height=500)

        self.entry_text_register_new_user = get_entry_text(self.register_new_user_window)
        self.entry_text_register_new_user.place(x=750, y=150)

        self.text_label_register_new_user = get_text_label(self.register_new_user_window, 'Please, \ninput username:')
        self.text_label_register_new_user.place(x=750, y=70)

        self.cap = cv2.VideoCapture(1)  # Reinitialize the camera
        self.add_webcam(self.capture_label)

        # Bind the close event to the register window
        self.register_new_user_window.protocol("WM_DELETE_WINDOW", self.cancel_register)

    def accept_register_new_user(self):
        name = self.entry_text_register_new_user.get("1.0", "end-1c").strip()
        if name:
            self.register_button.config(state=tk.DISABLED)
            self.cancel_button.config(state=tk.DISABLED)
            self.entry_text_register_new_user.config(state=tk.DISABLED)
            self.start_registration(name)
        else:
            msg_box('Error', 'Please enter a valid name.')

    def cancel_register(self):
        self.register_new_user_window.destroy()  # Destroy the registration window
        self.cap.release()  # Release the camera
        self.main_window.deiconify()  # Show the main window
        self.cap = cv2.VideoCapture(1)  # Reinitialize the camera
        self.add_webcam(self.webcam_label)  # Re-add the webcam to the main window

    def start_registration(self, name):
        person_folder = os.path.join(self.db_dir, name)
        os.makedirs(person_folder, exist_ok=True)

        count = 0
        end_time = datetime.now() + timedelta(seconds=5)

        def capture_frame():
            nonlocal count
            ret, frame = self.cap.read()
            if not ret:
                return

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)

            for (x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                face_img = frame[y:y + h, x:x + w]

                # Resize to 100x100
                face_img = cv2.resize(face_img, (100, 100))

                # Save the captured face
                face_path = os.path.join(person_folder, f'{count + 1}.jpg')
                cv2.imwrite(face_path, face_img)

                count += 1

            # Display countdown timer
            remaining_time = end_time - datetime.now()
            cv2.putText(frame, f'Remaining Time: {remaining_time.seconds}s', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            cv2.imshow('Registration', frame)

            if datetime.now() < end_time and count < 10:
                self.register_new_user_window.after(500, capture_frame)
            else:
                cv2.destroyWindow('Registration')
                msg_box('Success!', f'Registration complete for {name}.')
                log_event(name, 'register')
                
                self.cancel_register()

        capture_frame()

    def on_closing(self):
        self.cancel_webcam_update()
        self.cap.release()
        self.main_window.destroy()
        cv2.destroyAllWindows()

    def start(self):
        self.main_window.mainloop()

if __name__ == "__main__":
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    app = App()
    app.start()


2024-05-27 20:16:02.400 python[40812:1676622] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step


2024-05-27 20:23:32.771090: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


[[[0.8004378]]] zun yang chin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
[[[0.44579953]]] zun yang chin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
[[[0.8631288]]] zun yang chin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[[0.5825737]]] zun yang chin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[[0.78071094]]] zun yang chin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[[0.5974766]]] zun yang chin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[[0.27138874]]] zun yang chin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[[0.9577876]]] zun yang chin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[[0.5291567]]] zun yang chin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[[0.7243841]]] zun yang chin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
[[[0.67922676]]] zun yang chin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[[0.7584498]]] zun yang chin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[[0.69112253]]] zun yang chin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[[0.371997]]] zun yang chin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
[[[0.55731946]]] zun yang chin
1/1 ━━━━━━━━━

In [ ]:
# # to clear the attendance.csv, and verification_images_app
# import shutil
# import os

# for root, dirs, files in os.walk('./siamese_application_data/verification_images_app'):
#     for dir in dirs:
#         shutil.rmtree(os.path.join(root, dir))



# # opening the file with w+ mode truncates the file
# f = open("siamese_application_data/attendance.csv", "w+")
# f.close()
